# Logistic Regression Model


In [ ]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

# Preprocesado y modelado
# ==============================================================================
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import RandomizedSearchCV, KFold, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics

# Configuración matplotlib
# ==============================================================================
plt.rcParams['image.cmap'] = "bwr"
plt.rcParams['savefig.bbox'] = "tight"
style.use('ggplot') or plt.style.use('ggplot')

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')


Utilizaremos el dataset de cáncer de mama (breast cancer)

In [ ]:
data = load_breast_cancer()

In [ ]:
print(data.DESCR)

In [ ]:
print(data.keys())

In [ ]:
# Pasamos a un Data frame
df = pd.DataFrame(data.data, columns=data.feature_names)
# Añadimos una columna con la variable objetivo
df['target'] = data.target
# Mostramos las primeras filas
df.head()

In [ ]:
df.info()

Vemos como todas las variables de entradas son `float64`. Además, no faltan valores, todas las columnas tienen 569 valores.

In [ ]:
# Número de muestras por clase
# ==============================================================================
df.target.value_counts().sort_index()

Tenemos 212 muestras que no tienen cáncer y 357 que sí tienen cáncer. Aunque están un poco desbalanceadas las clases, lo vamos a dejar así.

In [ ]:
corr_Matrix = df.corr ()
f,ax = plt.subplots(figsize=(18, 18))
sns.heatmap (corr_Matrix, linewidths = 0.5, annot = True, fmt= '.1f',ax=ax)
plt.show ()

De la matriz de correlación vemos que las variables "_mean" y "_worst" se encuentran fuertemente correladas con la variable "target" y, además, también se encuentra fuertemente correladas entre ellas ("mean radius" tiene una correlación de 1.0 con "worst radio"). Por lo tanto, nos vamos a quedar con las variables "mean" que tengan al menos una correlación superior a 0.4 con la variable de salida ("target")

In [ ]:
label = []
for i in range (30):
    if np.abs(corr_Matrix.target[i]) < 0.4 or i>=10 :
        label.append (df.columns.values[i])
df.drop (labels = label, axis = 1, inplace = True)
df.head ()

In [ ]:
sns.pairplot(df, hue = "target")
plt.show()

In [ ]:
# Pasamos de dataframe a numpy para poder trabajar con sklearn
X = df.iloc [:, 0:6].values
y = df.iloc [:, 7].values
# dividimos las muestras en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.33, random_state=42, shuffle = True)

## LogisticRegression
Los parámetros más importantes de la implantación de sklearn (`LogisticRegression`) son:

- `penalty`: El tipo de aplicación de regularización. Sus valores pueden ser:{None, 'l2' (por defecto), 'l1', 'elascticnet'}
- `C`: (por defecto 1.0) Inverso de la fuerza de regularización; Valores más pequeños especifican una regularización más fuerte.
- `solver`: Algoritmo a utilizar en el problema de optimización. Sus valores pueden ser: {‘lbfgs’ (por defecto), ‘liblinear’, ‘newton-cg’, ‘newton-cholesky’, ‘sag’, ‘saga’}. Algunas consideraciones:

In [ ]:
# Probamos el modelo sin aplicar ninguna regularización y con los parámetros por efecto
# ==============================================================================
scaler = StandardScaler()
lr = LogisticRegression(penalty=None, random_state = 42)
#lr = LogisticRegression(random_state = 42)

pipe_scale_lr = Pipeline([
    ('scale', scaler),
    ('lr', lr)])

# Entrenamiento del modelo
# ==============================================================================
cv = KFold(n_splits=5, shuffle=True, random_state = 42 )
scores = cross_val_score(pipe_scale_lr, X_train, y_train, scoring='accuracy', cv = cv) #OJO!!! scoring=‘balanced_accuracy’
                                                                                       # ¿Qué diferencia cv=5?
print(f"All the accuracies are: {scores}")
print(f"And the average crossvalidation accuracy is: {scores.mean():.2f} +- {scores.std():.2f}")


Ahora procedemos a la búsqueda de los hiperparámteros

In [ ]:
param_grid = [{'lr__penalty': ['l1', 'l2', 'elascticnet'],
               'lr__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
               'lr__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}]


inner = KFold(n_splits=3, shuffle=True, random_state=42)

#budget = 40
# Cross-validation (3-fold) para la búsqueda de hiper-parámetros
clf = GridSearchCV (estimator  = pipe_scale_lr,
                    param_grid = param_grid,
                    scoring='accuracy', #OJO!!! scoring=‘balanced_accuracy’
                    cv=inner,
                    refit=True,
                    n_jobs=-1,
                    verbose=1,
                    return_train_score=True)

np.random.seed(42)

clf.fit(X=X_train, y=y_train)

In [ ]:
resultados = pd.DataFrame(clf.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head()

In [ ]:
clf.best_params_, clf.best_score_

Al poner el parámetro `refit=True` se reentrena el modelo indicando los valores óptimos en sus argumentos. Este reentrenamiento se hace automáticamente y el modelo resultante se encuentra almacenado en `.best_estimator_`.

In [ ]:
# Información del modelo
# ==============================================================================
modelo_final = clf.best_estimator_
print("Intercept:", modelo_final['lr'].intercept_)
print("Coeficientes:", list(zip(df.columns, modelo_final['lr'].coef_.flatten(), )))
print("Accuracy de test:", modelo_final.score(X_test, y_test))

In [ ]:
y_test_pred = modelo_final.predict(X_test)
result = metrics.classification_report(y_test, y_test_pred)
print("Classification Report:",)
print (result)

In [ ]:
# Creates a confusion matrix
cm = metrics.confusion_matrix(y_test, y_test_pred)
accuracy = metrics.accuracy_score(y_test, y_test_pred)
# Transform to df for easier plotting
cm_df = pd.DataFrame(cm,
                     index = ['Benigno','Maligno'],
                     columns = ['Benigno','Maligno'])
plt.figure(figsize=(5.5,4))
sns.heatmap(cm_df, annot=True)
plt.title('Accuracy:{0:.3f}'.format(accuracy))
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
#Obtenemos las curva ROC y el área bajo la curva (AUC)

probs = modelo_final.predict_proba(X_test)[:, 1]

auc = metrics.roc_auc_score(y_test, probs)
fpr, tpr, thresholds = metrics.roc_curve(y_test, probs)

plt.figure(figsize=(8, 5))
plt.plot(fpr, tpr, label=f'AUC  = {auc:.2f}')
plt.plot([0, 1], [0, 1], color='blue', linestyle='--', label='Baseline')
plt.title('Curva ROC', size=20)
plt.xlabel('Falsos Positivos', size=14)
plt.ylabel('Verdaderos Positivos', size=14)
plt.legend();

In [ ]:
# Entrenamos con todos los datos para el modelo final
_ = modelo_final.fit(X,y)

# Selección de características mediante métododos de filtrado

## Basados en la varianza

In [ ]:
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold=0.01) # Umbral de varianza

# Dividimos los df de las variables de entrada y la variable objetivo
df_t = df['target']
df_f = df.drop('target',axis=1)

sel = selector.fit(df_f)
sel_index = sel.get_support()
df_vt = df_f.iloc[:, sel_index]
print(df_vt.columns)

## Mutual info
Calcula el valor de información mutua de cada una de las variables independientes con respecto a la variable dependiente y selecciona las que tienen mayor ganancia de información (al estilo de la ganancia de información de los árboles de decisión). En otras palabras, básicamente mide la dependencia de las características con el valor objetivo. Cuanto mayor sea la puntuación, mayor será la dependencia.

In [ ]:
from sklearn.feature_selection import mutual_info_classif

threshold = 4  # the number of most relevant features
high_score_features = []
feature_scores = mutual_info_classif(df_f, df_t, random_state=42)
for score, f_name in sorted(zip(feature_scores, df_f.columns), reverse=True)[:threshold]:
    print(f_name, score)
    high_score_features.append(f_name)
df_mic = df_f[high_score_features]
print(df_mic.columns)

También podemos seleccionar, por ejemplo, aquellas variable cuya IM sea mayor que un valor (por ejemplo 0.2). Además, como ejemplo, voy a buscar el mejor clasificador usando sólo las variables seleccionadas. Esto mismo lo podemos hacer con el resto de métodos Filter.

In [ ]:
mi_score_selected_index = np.where(feature_scores >0.2)[0]
df_mic2 = df.iloc[:,mi_score_selected_index]
X_train2,X_test2,y_train,y_test = train_test_split(df_mic2,df_t,test_size=0.33,
                                                     random_state=42,
                                                     shuffle = True)
# Recordar que "clf" hace una búsqueda para los mejores parámetros
np.random.seed(42)
clf.fit(X=X_train2, y=y_train)

In [ ]:
clf.best_params_, clf.best_score_

In [ ]:
y_test_pred = clf.predict(X_test2)
result = metrics.classification_report(y_test, y_test_pred)
print("Classification Report with filtered features:",)
print (result)

## F_classif
Utiliza el test f de ANOVA para las características y sólo tiene en cuenta la dependencia lineal, a diferencia de la selección de características basada en la información mutua, que puede capturar cualquier tipo de dependencia estadística. Observar que las puntuaciones obtenidas por los distintos métodos son totalmente diferentes. No os quedeis en este punto, cada método ordena internamente la importancia de las características y devuelve las mejores.


In [ ]:
from sklearn.feature_selection import f_classif
threshold = 4 # the number of most relevant features

high_score_features = []
feature_scores = f_classif(df_f, df_t)[0]
for score, f_name in sorted(zip(feature_scores, df_f.columns), reverse=True)[:threshold]:
    print(f_name, score)
    high_score_features.append(f_name)
df_fc = df_f[high_score_features]
print(df_fc.columns)

## Chi2
Esta clase es en realidad un enfoque más general en comparación con las clases antes mencionadas, ya que toma un parámetro adicional de función de puntuación que establece qué función utilizar en la selección de características. Por lo tanto, se puede considerar como una especie de envoltorio. También podemos utilizar f_classif o mutual_info_class_if dentro de este objeto. Por otra parte, se utiliza típicamente con la función chi2. Este objeto devuelve también los p-values de cada característica según la función de puntuación elegida.

El test chi2 mide la dependencia entre variables estocásticas, por lo que podemos eliminar las características que tienen más probabilidades de ser independientes del objetivo utilizando esta función. Sirve básicamente para evaluar si la diferencia entre dos grupos separados de muestras no negativas se debe al azar o no.

El test chi2 parte de la hipótesis nula de que dos variables son independientes y de la hipótesis alternativa de que dos variables son dependientes, como la mayoría de las pruebas estadísticas. Mediante el test chi2, se calculan los p-values de cada característica en relación con el objetivo. De forma sencilla, p es la probabilidad de que dos variables sean independientes. Lo que se busca es determinar si las características que dependen del objetivo, es decir, rechazan la hipótesis nula. Por este motivo, seleccionamos las características que suelen tener un p-value inferior a 0,05. El valor umbral de 0,05 es sólo un comportamiento común, puedes establecer valores umbral más pequeños como 0,01 para estar más seguro de que dos grupos son dependientes.


In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
threshold = 4  # the number of most relevant features

skb = SelectKBest(score_func=chi2, k=threshold)
sel_skb = skb.fit(df_f, df_t)
sel_skb_index = sel_skb.get_support()
df_skb = df_f.iloc[:, sel_skb_index]
print('p_values', sel_skb.pvalues_)
print(df_skb.columns)